# Chapter 4. Iterators and Generators

Iteration is one of Python’s strongest features.  
At a high level, you might simply view iteration as a way to process items in a sequence.  
However, there is so much more that is possible, such as creating your own iterator objects, applying useful iteration patterns in the itertools module, making generator functions, and so forth.  
This chapter aims to address common problems involving iteration.

## [Manually Consuming an Iterator](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_manually_consuming_an_iterator)

### Problem

You need to process items in an iterable, but for whatever reason, you can't or don't want to use a for loop.

### Solution

To manually consume an iterable, use the `next()` function and write your code to catch the `StopIteration` exception.  
This example manually reads lines from a file:

In [104]:
with open('python_ipsum.txt') as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration:
        pass
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.
http://pythonipsum.com/

Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bd

Normally, `StopIteration` is used to signal the end of iteration.  
However, if you're using `next()` manually (as shown), you can also instruct it to return a terminating value, such as `None`, instead.

In [105]:
with open('python_ipsum.txt') as f:
    while True:
        line = next(f, None)
        if line is None:
            break
        print(line, end='')
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.
http://pythonipsum.com/

Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bd

### Discussion

In most cases, the `for` statement is used to consume an iterable.  
However, every now and then, a problem calls for more precise control over the underlying iteration mechanism.  
Thus, it is useful to know what actually happens.  
The following example illustrates the basic mechanics of what happens during iteration:

In [106]:
items = [1, 2, 3]
# Get the iterator:
it = iter(items)  # Invokes items.__iter__()
# Run the iterator:
next(it)          # Invokes it.__next__()

1

In [107]:
next(it)

2

In [108]:
next(it)

3

Subsequent recipes in this chapter expand on iteration techniques, and knowledge of the basic iterator protocol is assumed.  
Be sure to tuck this first recipe away in your memory.

## [Delegating Iteration](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#delegate_iteration)

### Problem

You have built a custom container object that internally holds a list, tuple, or some other iterable.  
You would like to make iteration work with your new container.

### Solution

Typically, all you need to do is define an `__iter__()` method that delegates iteration to the internally held container.

In [109]:
class Node:
    
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)

Now for a demonstration:

In [110]:
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    for ch in root:
        print(ch)

Node(1)
Node(2)


In this code, the `__iter__()` method simply forwards the iteration request to the internally held `_children` attribute.

### Discussion

Python’s iterator protocol requires `__iter__()` to return a special iterator object that implements a `__next__()` method to carry out the actual iteration.  
If all you are doing is iterating over the contents of another container, you don’t really need to worry about the underlying details of how it works.  
All you need to do is to forward the iteration request along.  
The use of the `iter()` function here is a bit of a shortcut that cleans up the code.  
`iter(s)` simply returns the underlying iterator by calling `s.__iter__()`, much in the same way that `len(s)` invokes `s.__len__()`.

## [Creating New Iteration Patterns with Generators](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_problem_59)

### Problem

You want to implement a custom iteration pattern that’s different than the usual built-in functions (e.g., `range()`, `reversed()`, etc.).

### Solution

If you want to implement a new kind of iteration pattern, define it using a generator function.  
Here’s a generator that produces a range of floating-point numbers:

In [111]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

To use such a function, you iterate over it using a for loop or use it with some other function that consumes an iterable (e.g., `sum()`, `list()`, etc.).

In [112]:
for n in frange(0, 4, 0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [113]:
# I want to try this with a list comprehension:
comp = [n for n in frange(0, 4, 0.5)]
print(comp)

[0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]


In [114]:
list(frange(0, 1, 0.125))

[0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]

### Discussion

The mere presence of the yield statement in a function turns it into a generator.  
Unlike a normal function, a generator only runs in response to iteration.  
Here’s an experiment you can try to see the underlying mechanics of how such a function works:

In [115]:
def countdown(n):
    print('Starting to count from', n)
    while n > 0:
        yield n
        n -= 1
    print('Done, sucka!')

Create the generator and notice that no output appears:

In [116]:
c = countdown(3)
c

<generator object countdown at 0x10c34ff68>

Run the program to the first `yield` and emit a value.

In [117]:
next(c)

Starting to count from 3


3

Run the program to the next `yield` and repeat:

In [118]:
next(c)

2

In [119]:
next(c)

1

The key feature is that a generator function only runs in response to "next" operations carried out in iteration.  
Once a generator function returns, iteration stops.  
However, the `for` statement that’s usually used to iterate takes care of these details, so you don’t normally need to worry about them.

## [Implementing the Iterator Protocol](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_implementing_the_iterator_protocol)

### Problem

You are building custom objects on which you would like to support iteration, but would like an easy way to implement the iterator protocol.

### Solution

By far, the easieast way to implement iteration on an object is to use a generator function.  
In "Delegating Iteration", a `Node` class was presented for representing tree structures.  
Perhaps you want to implement an iterator that traverses nodes in a depth-first pattern.

In [120]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
    # https://www.pythoncentral.io/what-is-the-difference-between-__str__-and-__repr__-in-python/
    # https://docs.python.org/3.6/reference/datamodel.html#object.__repr__
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    # https://docs.python.org/3.6/library/stdtypes.html?highlight=__iter__#iterator.__iter__    
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()
            
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In this code, the `depth_first()` method is fairly straightforward.  
It first yields itself and then iterates over each child yielding the items produced by the child's `depth_first()` method (using `yield from`).

### Discussion

Python’s iterator protocol requires `__iter__()` to return a special iterator object that implements a `__next__()` operation and uses a `StopIteration` exception to signal completion.  
However, implementing such objects can often be a messy affair.  
For example, the following code shows an alternative implementation of the `depth_first()` method using an associated iterator class:

In [121]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, other_node):
        self._children.append(other_node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        return DepthFirstIterator(self)
    
    
class DepthFirstIterator(object):
    """
    Depth-first traversal
    """
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
        
    def __iter__(self):
        return self
    
    def __next__(self):
        # Create an iterator for the children.
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        
        # If processing a child, return its next item.
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
        # Advance to the next child and begin iteration.
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)
        
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))

    for ch in root.depth_first():
        print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


The `DepthFirstIterator` class works in the same way as the generator version, but it’s a mess because the iterator has to maintain a lot of complex states about where it is in the iteration process.  
Frankly, nobody likes to write mind-bending code like that.  
Define your iterator as a generator and be done with it.

## [Iterating in Reverse](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_iterating_in_reverse)

### Problem

You want to iterate in reverse over a sequence.

### Solution

Use the built-in `reversed()` function.

In [122]:
a = [1, 2, 3, 4]
for x in reversed(a):
    print(x)

4
3
2
1


Reversed iteration only works if the object in question has a size that can be determined or if the object implements a `__reversed__()` special method.  
If neither of these can be satisfied, you’ll have to convert the object into a list first.

In [123]:
# Print a file backwards:
f = open('python_ipsum.txt')
for line in reversed(list(f)):
    print(line, end='')
f.close()

Method integration coroutine unit import cython implicit django python. Return raspberrypi functools pypi method gil dunder. Dunder raspberrypi decorator list reduce numpy. Future raspberrypi coroutine bdfl python egg gevent python functools descriptor.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bdfl. Python integration generator pypy script. Gevent integration generator pypy lambda jinja decorator arduino dict zip.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs ra

Be aware that turning an iterable into a list as shown could consume a lot of memory if that iterable is large.

### Discussion

Many programmers don’t realize that reversed iteration can be customized on user-defined classes if they implement the `__reversed__()` method.

In [124]:
class Countdown:
    def __init__(self, start):
        self.start = start
        
    # Forward iterator:
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
            
    # Reverse iterator:
    def __reversed__self():
        n = 1
        while n <= self.start:
            yield n
            n += 1

Defining a reversed iterator makes the code much more efficient, as it’s no longer necessary to pull the data into a list and iterate in reverse on the list.

## [Defining Generator Functions with Extra State](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_defining_generator_functions_with_extra_state)

### Problem

You would like to define a generator function, but it involves extra state that you would like to expose to the user somehow.

### Solution

If you want a genereator to expose extra state to the user, don't forget that you can easily implement it as a class, putting the generator function code in the `__iter__()` method:

In [125]:
from collections import deque

class Linehistory:
    
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for line_number, line in enumerate(self.lines, 1):
            self.history.append((line_number, line))
            yield line

    def clear(self):
        self.history.clear()

To use this class, you would treat it like a normal generator function.  
However, since it creates an instance, you can access internal attributes, such as the `history` attribute or the `clear()` method.

In [126]:
with open('python_ipsum.txt') as f:
    lines = Linehistory(f)
    for line in lines:
        if 'python' in line:
            for line_number, hline in lines.history:
                print('{}:{}'.format(line_number, hline), end='')
f.close()

1:Python Ipsum: Your source for Python-flavored placeholder text.
2:http://pythonipsum.com/
4:Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.
5:
6:Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.
6:Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python inte

### Discussion 

With generators, it is easy to fall into a trap of trying to do everything with functions alone.  
This can lead to rather complicated code if the generator function needs to interact with other parts of your program in unusual ways (exposing attributes, allowing control via method calls, etc.).  
If this is the case, just use a class definition, as shown.  
Defining your generator in the `__iter__()` method doesn’t change anything about how you write your algorithm.  
The fact that it’s part of a class makes it easy for you to provide attributes and methods for users to interact with. 

One potential subtlety with the method shown is that it might require an extra step of calling `iter()` if you are going to drive iteration using a technique other than a for loop.

In [127]:
f = open('python_ipsum.txt')
lines = Linehistory(f)
it = iter(lines)
print(next(it))
print(next(it))
print(next(it))
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.

http://pythonipsum.com/





## [Taking a Slice of an Iterator](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#takingasliceofaniterator)